# ICCNet Training Script

This script trains the convolution neural network to recognize $s_\text{cvx}$ values for every number of bases employed in compressive tomography, and later predicts the values with test examples. The complete simulated data consist of those from both statistically noiseless and noisy cases of ACT and random Haar schemes (4 batches). Each batch consists of $m=5000$ training datasets generated from a separate MATLAB script that simulates the ACT and random Haar schemes on 4-qubit systems ($d=2^4=16$).

### Loads packages.

In [1]:
import scipy.io as spi
import os

import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
import itertools
import json
import h5py

from keras.layers import Concatenate, Dense, Dropout, Conv2D, MaxPooling2D, Flatten, merge, Input, ZeroPadding2D, Activation, Add, AveragePooling2D
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.models import Sequential, load_model, Model
from keras import optimizers, regularizers
from keras import backend as K
from keras.layers.normalization import BatchNormalization

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

/home/ys_teo/ysteo-tensorflow_old/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ys_teo/ysteo-tensorflow_old/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ys_teo/ysteo-tensorflow_old/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ys_teo/ysteo-tensorflow_old/lib/python3.5/s

### Defines work parameters

In [2]:
d=16 # Hilbert-space dimension
m=8000 # Number of training datasets
num_classes=1 # Set to 1 by default since we are fitting a single continuous output value

# Defines the working directory containing all generated MATLAB MAT training data files
work_dir='../training/'+str(m)+'_perf_noisy_ex/'

# Defines ICCNet model storage directory
stor_dir=work_dir+'ICCNet_trained_files/'

### Training commences

In [ ]:
# Loops over different number of bases "Kbas" for training ICCNet

for l in np.linspace(2,10,9):

    Kbas=int(l)

    # Loads the relevant input matrix for the defined Kbas.
    X = spi.loadmat(work_dir+'X_'+str(Kbas)+'.mat')['X']
    X = X[:,0:Kbas*(d**2+d)]

    # Loads the SCVX values.
    y = spi.loadmat(work_dir+'y_'+str(Kbas)+'.mat')['y']
    y = y[:,0].reshape([2*m,1])
    y = np.abs(y)

    # Logarithmizes SCVX values.
    y = -np.log10(np.abs(y))
    y = y+(y>10)*(10-y)
    y = np.abs(y)
    y = y/10

    if os.path.isdir(stor_dir)==False:
        os.mkdir(stor_dir)

    # --------------------------------------------------------------------------
    # Data preprocessing and splitting to 'train', 'validation' and 'test' sets.
    # --------------------------------------------------------------------------  

    # Splits the full dataset into training, validation and test subsets.
    x_train0, x_rest, y_train, y_rest = train_test_split(X, y, test_size=0.2, shuffle=True)
    x_test, x_val, y_test, y_val = train_test_split(x_rest, y_rest, test_size=0.5, shuffle=True)

    # Performs standard data normalization and scaling.
    x_train = preprocessing.StandardScaler().fit(x_train0).transform(x_train0)
    x_train = x_train.astype('float32')
    x_test = preprocessing.StandardScaler().fit(x_train0).transform(x_test)
    x_test = x_test.astype('float32')
    x_val = preprocessing.StandardScaler().fit(x_train0).transform(x_val)
    x_val = x_val.astype('float32')  

    # ---------------------------------------------
    # ICCNet model definition and parametrization.
    # ---------------------------------------------

    # Defines gradient optimization batch size and training epochs
    batch_size = 256
    epochs = 1000

    tf.keras.initializers.he_normal()

    # Converts inputs to single-channel 2D images
    img_rows = int(np.ceil(np.power(x_train.shape[1],0.5)))
    img_cols = img_rows
    x_train = np.concatenate((x_train,np.zeros((x_train.shape[0],img_rows*img_cols-x_train.shape[1]))),axis=1)
    x_test = np.concatenate((x_test,np.zeros((x_test.shape[0],img_rows*img_cols-x_test.shape[1]))),axis=1)
    x_val = np.concatenate((x_val,np.zeros((x_val.shape[0],img_rows*img_cols-x_val.shape[1]))),axis=1)

    K.clear_session()
    x_train=x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test=x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    x_val=x_val.reshape(x_val.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1) 
        
    # Sets up neural network
    X_input = Input(input_shape) 

    Xnn = Conv2D(4, kernel_size = (3, 3), padding='same', kernel_initializer = 'he_uniform')(X_input)
    Xnn = BatchNormalization()(Xnn)
    Xnn = Activation('relu')(Xnn)
    Xnn = Dropout(0.2)(Xnn)
    Xnn = MaxPooling2D(pool_size=(2,2),strides=2)(Xnn)    

    Xnn = Conv2D(8, kernel_size = (3, 3), padding='same', kernel_initializer = 'he_uniform')(Xnn)
    Xnn = BatchNormalization()(Xnn)
    Xnn = Activation('relu')(Xnn)
    Xnn = Dropout(0.3)(Xnn)
    Xnn = MaxPooling2D(pool_size=(2,2),strides=2)(Xnn)

    Xnn = Conv2D(16, kernel_size = (3, 3), padding='same', kernel_initializer = 'he_uniform')(Xnn)
    Xnn = BatchNormalization()(Xnn)
    Xnn = Activation('relu')(Xnn)
    Xnn = Dropout(0.5)(Xnn)
    Xnn = MaxPooling2D(pool_size=(2,2),strides=2)(Xnn)

    Xnn = Conv2D(32, kernel_size = (3,3), padding='same', kernel_initializer = 'he_uniform')(Xnn)
    Xnn = BatchNormalization()(Xnn)
    Xnn = Activation('relu')(Xnn)
    Xnn = Dropout(0.7)(Xnn)

    Xnn = Conv2D(32, kernel_size = (3,3), padding='same', kernel_initializer = 'he_uniform')(Xnn)
    Xnn = BatchNormalization()(Xnn)
    Xnn = Activation('relu')(Xnn)
    Xnn = Dropout(0.9)(Xnn)

    Xnn = Flatten()(Xnn)

    Xnn = Dense(num_classes,activation='sigmoid', kernel_initializer = 'he_uniform')(Xnn)

    model = Model(inputs = X_input, outputs = Xnn)

    model.summary()

    # Sets mean absolute error as the cost function and optimizer as NADAM.
    model.compile(loss='mae',optimizer='nadam')

    # -----------------
    # Network training.
    # -----------------

    # Sets up the scoreboard and checkpoint monitor to record training results.    
    checkpointer = ModelCheckpoint(filepath=stor_dir+"model_ICCNet_K_"+str(Kbas)+".h5",
                                   verbose=0, monitor='val_loss', mode='min', save_best_only=True)
    tensorboard = TensorBoard(log_dir='./logs',
                              histogram_freq=0,
                              write_graph=True,
                              write_images=True)

    # Trains ICCNet with training and validation data subsets and records all results.
    history = model.fit(x_train, y_train,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(x_val, y_val),
                        verbose=1,
                        callbacks=[checkpointer, tensorboard]).history

    with open(stor_dir+"model_ICCNet_K_"+str(Kbas)+"_hist.txt", 'w') as outfile:  
        json.dump(history, outfile)

    # Evaluates the trained model with test data subsets.
    score=model.evaluate(x_test,y_test,verbose=0)

    print('Test loss:',score)

    # Stores all split data subsets into a dictionary and saves it into a MATLAB cell file.
    Xy_dict = {'x_val': x_val, 'y_val': y_val, 'x_test': x_test, 'y_test': y_test, 'x_train0': x_train0, 'y_train': y_train}
    spi.savemat(stor_dir+"model_ICCNet_K_"+str(Kbas)+"_Xy.mat", Xy_dict)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 24, 24, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 4)         40        
_________________________________________________________________
batch_normalization_1 (Batch (None, 24, 24, 4)         16        
_________________________________________________________________
activation_1 (Activation)    (None, 24, 24, 4)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 24, 24, 4)         0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 4)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 8)         296       
__________

12800/12800 [==============================] - 1s 64us/step - loss: 0.0110 - val_loss: 0.0075
Epoch 42/1000
12800/12800 [==============================] - 1s 68us/step - loss: 0.0109 - val_loss: 0.0070
Epoch 43/1000
12800/12800 [==============================] - 1s 68us/step - loss: 0.0106 - val_loss: 0.0074
Epoch 44/1000
12800/12800 [==============================] - 1s 64us/step - loss: 0.0105 - val_loss: 0.0075
Epoch 45/1000
12800/12800 [==============================] - 1s 65us/step - loss: 0.0104 - val_loss: 0.0068
Epoch 46/1000
12800/12800 [==============================] - 1s 66us/step - loss: 0.0102 - val_loss: 0.0067
Epoch 47/1000
12800/12800 [==============================] - 1s 68us/step - loss: 0.0099 - val_loss: 0.0070
Epoch 48/1000
12800/12800 [==============================] - 1s 63us/step - loss: 0.0100 - val_loss: 0.0064
Epoch 49/1000
12800/12800 [==============================] - 1s 66us/step - loss: 0.0098 - val_loss: 0.0071
Epoch 50/1000
12800/12800 [===============

Epoch 117/1000
12800/12800 [==============================] - 1s 68us/step - loss: 0.0041 - val_loss: 0.0037
Epoch 118/1000
12800/12800 [==============================] - 1s 66us/step - loss: 0.0041 - val_loss: 0.0037
Epoch 119/1000
12800/12800 [==============================] - 1s 67us/step - loss: 0.0041 - val_loss: 0.0037
Epoch 120/1000
12800/12800 [==============================] - 1s 65us/step - loss: 0.0041 - val_loss: 0.0037
Epoch 121/1000
12800/12800 [==============================] - 1s 68us/step - loss: 0.0041 - val_loss: 0.0036
Epoch 122/1000
12800/12800 [==============================] - 1s 67us/step - loss: 0.0041 - val_loss: 0.0036
Epoch 123/1000
12800/12800 [==============================] - 1s 67us/step - loss: 0.0040 - val_loss: 0.0037
Epoch 124/1000
12800/12800 [==============================] - 1s 64us/step - loss: 0.0040 - val_loss: 0.0036
Epoch 125/1000
12800/12800 [==============================] - 1s 66us/step - loss: 0.0040 - val_loss: 0.0037
Epoch 126/1000
1280

12800/12800 [==============================] - 1s 63us/step - loss: 0.0038 - val_loss: 0.0035
Epoch 193/1000
12800/12800 [==============================] - 1s 65us/step - loss: 0.0038 - val_loss: 0.0035
Epoch 194/1000
12800/12800 [==============================] - 1s 64us/step - loss: 0.0038 - val_loss: 0.0035
Epoch 195/1000
12800/12800 [==============================] - 1s 68us/step - loss: 0.0038 - val_loss: 0.0036
Epoch 196/1000
12800/12800 [==============================] - 1s 69us/step - loss: 0.0038 - val_loss: 0.0035
Epoch 197/1000
12800/12800 [==============================] - 1s 64us/step - loss: 0.0038 - val_loss: 0.0035
Epoch 198/1000
12800/12800 [==============================] - 1s 66us/step - loss: 0.0038 - val_loss: 0.0034
Epoch 199/1000
12800/12800 [==============================] - 1s 66us/step - loss: 0.0038 - val_loss: 0.0035
Epoch 200/1000
12800/12800 [==============================] - 1s 66us/step - loss: 0.0038 - val_loss: 0.0034
Epoch 201/1000
12800/12800 [======

12800/12800 [==============================] - 1s 66us/step - loss: 0.0037 - val_loss: 0.0034
Epoch 268/1000
12800/12800 [==============================] - 1s 67us/step - loss: 0.0038 - val_loss: 0.0033
Epoch 269/1000
12800/12800 [==============================] - 1s 67us/step - loss: 0.0038 - val_loss: 0.0033
Epoch 270/1000
12800/12800 [==============================] - 1s 65us/step - loss: 0.0037 - val_loss: 0.0034
Epoch 271/1000
12800/12800 [==============================] - 1s 68us/step - loss: 0.0037 - val_loss: 0.0033
Epoch 272/1000
12800/12800 [==============================] - 1s 69us/step - loss: 0.0037 - val_loss: 0.0034
Epoch 273/1000
12800/12800 [==============================] - 1s 68us/step - loss: 0.0038 - val_loss: 0.0034
Epoch 274/1000
12800/12800 [==============================] - 1s 65us/step - loss: 0.0037 - val_loss: 0.0034
Epoch 275/1000
12800/12800 [==============================] - 1s 68us/step - loss: 0.0038 - val_loss: 0.0034
Epoch 276/1000
12800/12800 [======

...

12800/12800 [==============================] - 1s 76us/step - loss: 0.1007 - val_loss: 0.1191
Epoch 993/1000
12800/12800 [==============================] - 1s 79us/step - loss: 0.1010 - val_loss: 0.1137
Epoch 994/1000
12800/12800 [==============================] - 1s 78us/step - loss: 0.1015 - val_loss: 0.1219
Epoch 995/1000
12800/12800 [==============================] - 1s 78us/step - loss: 0.1008 - val_loss: 0.1261
Epoch 996/1000
12800/12800 [==============================] - 1s 78us/step - loss: 0.1019 - val_loss: 0.1280
Epoch 997/1000
12800/12800 [==============================] - 1s 74us/step - loss: 0.1016 - val_loss: 0.1197
Epoch 998/1000
12800/12800 [==============================] - 1s 77us/step - loss: 0.1020 - val_loss: 0.1201
Epoch 999/1000
12800/12800 [==============================] - 1s 79us/step - loss: 0.1009 - val_loss: 0.1185
Epoch 1000/1000
12800/12800 [==============================] - 1s 79us/step - loss: 0.1012 - val_loss: 0.1197
Test loss: 0.12333086743950844
__

12800/12800 [==============================] - 1s 87us/step - loss: 0.1055 - val_loss: 0.1140
Epoch 33/1000
12800/12800 [==============================] - 1s 81us/step - loss: 0.1047 - val_loss: 0.1182
Epoch 34/1000
12800/12800 [==============================] - 1s 84us/step - loss: 0.1030 - val_loss: 0.1069
Epoch 35/1000
12800/12800 [==============================] - 1s 84us/step - loss: 0.1024 - val_loss: 0.1064
Epoch 36/1000
12800/12800 [==============================] - 1s 84us/step - loss: 0.1014 - val_loss: 0.1128
Epoch 37/1000
12800/12800 [==============================] - 1s 86us/step - loss: 0.1021 - val_loss: 0.1128
Epoch 38/1000
12800/12800 [==============================] - 1s 86us/step - loss: 0.1018 - val_loss: 0.1207
Epoch 39/1000
12800/12800 [==============================] - 1s 83us/step - loss: 0.0999 - val_loss: 0.1227
Epoch 40/1000
12800/12800 [==============================] - 1s 83us/step - loss: 0.0990 - val_loss: 0.1239
Epoch 41/1000
12800/12800 [===============

Epoch 108/1000
12800/12800 [==============================] - 1s 86us/step - loss: 0.0873 - val_loss: 0.0845
Epoch 109/1000
12800/12800 [==============================] - 1s 86us/step - loss: 0.0869 - val_loss: 0.0875
Epoch 110/1000
12800/12800 [==============================] - 1s 81us/step - loss: 0.0865 - val_loss: 0.0905
Epoch 111/1000
12800/12800 [==============================] - 1s 86us/step - loss: 0.0869 - val_loss: 0.0953
Epoch 112/1000
12800/12800 [==============================] - 1s 83us/step - loss: 0.0875 - val_loss: 0.0867
Epoch 113/1000
12800/12800 [==============================] - 1s 85us/step - loss: 0.0864 - val_loss: 0.0878
Epoch 114/1000
12800/12800 [==============================] - 1s 87us/step - loss: 0.0868 - val_loss: 0.0881
Epoch 115/1000
12800/12800 [==============================] - 1s 83us/step - loss: 0.0873 - val_loss: 0.0929
Epoch 116/1000
12800/12800 [==============================] - 1s 86us/step - loss: 0.0867 - val_loss: 0.0932
Epoch 117/1000
1280

12800/12800 [==============================] - 1s 87us/step - loss: 0.0844 - val_loss: 0.0871
Epoch 184/1000
12800/12800 [==============================] - 1s 85us/step - loss: 0.0855 - val_loss: 0.0836
Epoch 185/1000
12800/12800 [==============================] - 1s 87us/step - loss: 0.0854 - val_loss: 0.0797
Epoch 186/1000
12800/12800 [==============================] - 1s 86us/step - loss: 0.0855 - val_loss: 0.0875
Epoch 187/1000
12800/12800 [==============================] - 1s 85us/step - loss: 0.0854 - val_loss: 0.0886
Epoch 188/1000
12800/12800 [==============================] - 1s 86us/step - loss: 0.0848 - val_loss: 0.0962
Epoch 189/1000
12800/12800 [==============================] - 1s 82us/step - loss: 0.0851 - val_loss: 0.0894
Epoch 190/1000
12800/12800 [==============================] - 1s 86us/step - loss: 0.0848 - val_loss: 0.0848
Epoch 191/1000
12800/12800 [==============================] - 1s 84us/step - loss: 0.0853 - val_loss: 0.0877
Epoch 192/1000
12800/12800 [======

12800/12800 [==============================] - 1s 82us/step - loss: 0.0841 - val_loss: 0.0856
Epoch 259/1000
12800/12800 [==============================] - 1s 87us/step - loss: 0.0844 - val_loss: 0.0853
Epoch 260/1000
12800/12800 [==============================] - 1s 86us/step - loss: 0.0841 - val_loss: 0.0835
Epoch 261/1000
 4864/12800 [==========>...................] - ETA: 0s - loss: 0.0858